## Importing libraries

In [0]:
import os
import tensorflow as tf

import shutil

import read_data as rd
import matplotlib.pyplot as plt
from model import gender_classifier,wce_loss

tf.reset_default_graph()

## Setting up directories and downloading the data

In [0]:
""" First downlaod the images and associated labels"""
dataset_path = 'data'
if os.path.exists(dataset_path):
    found = 1
else:
    os.mkdir(os.path.join(os.getcwd(),dataset_path))
    
model_dir = '/kernel_log'
if os.path.exists(model_dir):
    found = 1
else:
    os.mkdir(os.path.join(os.getcwd(),model_dir))

rd.download_extract(dataset_path)
dataset = rd.Dataset(dataset_path)

## Building computational graph for classifier, cost function and performance metrics (accuracy)

In [0]:
px= tf.placeholder(tf.float32, [None,28,28,3]) ## a placeholder for images
py= tf.placeholder(tf.float32, [None,2])  ## a placeholder for binary labels
ptr = tf.placeholder(tf.bool,shape=None) ## placeholder for a boolean which tells the graph if we are training or inferring labels for new data

## inference graph
pr = gender_classifier(px,ptr)

## cost function
cost = wce_loss(py,pr)

## optimizer
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

## Getting class number from inference    
prediction = tf.argmax(pr, 1)

## performance metrics (accuracy)
correct_answer = tf.argmax(py, 1)
equality = tf.equal(correct_answer,prediction)
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

## Training the classifier

In [0]:
batch_size = 32 ## batch size while training
num_train_batches = int(np.floor(dataset.X_train.shape[0]/batch_size)) ## number of batches in training data

## training session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) ## initialize graph

    for epochn in range(1): ## change the number of epochs for more training
        for batchn in range(num_train_batches): #iterate through all batches
            ## beginning and end idices of a batch
            in1 = batchn*batch_size
            in2 = in1 + batch_size

            ## get the training batch
            xb_train,yb_train = get_batch(dataset.X_train,dataset.Y_train,in1,in2,batch_size)

            ## infer labels and calculate loss for current training batch
            pred = sess.run(pr,feed_dict = {px : xb_train,ptr:True})
            batch_loss = sess.run(cost,feed_dict={py:yb_train,pr:pred})

            ## Check how the model performs on validation data after every 100th batch
            if (batchn%100)==0:
                nval = dataset.X_validate.shape[0]
                nbval = np.floor(nval/500)
                in1 = int((nbval-2)*500)
                in2 = in1 + 500
                xb_val,yb_val = get_batch(dataset.X_validate,dataset.Y_validate,in1,in2,500)
                pred = sess.run(pr,feed_dict = {px : xb_val,ptr:False})
                acc = sess.run(accuracy,feed_dict={py:yb_val,pr:pred})
                print('loss:'+str(batch_loss)+', accuracy: '+str(acc))

## Getting predictions for unseen data

In [0]:
## Get a random batch from the data. This can be replaced by data batch of your own 
data_batch, indices = dataset.get_image_batch(16) 

# Infer the classes for each image in the batch
gender_class = sess.run(prediction,feed_dict={px:data_batch,ptr:False})